<a href="https://colab.research.google.com/github/siddharth101/Double_suspension/blob/main/spring_block.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [205]:
from sympy import zeros, symbols
from sympy import symbols, Matrix, solve, simplify
from sympy import Matrix
from sympy.physics.mechanics import Body, PinJoint, SphericalJoint, PlanarJoint, PrismaticJoint, JointsMethod, inertia
from sympy.physics.mechanics import dynamicsymbols
from sympy import Symbol
from sympy.physics.mechanics import dynamicsymbols, ReferenceFrame, outer
from sympy.physics.mechanics import ReferenceFrame, Point, RigidBody
from sympy.physics.mechanics import kinetic_energy, potential_energy, Point, Particle

import sympy as smp
import numpy as np
import matplotlib.pyplot as plt
import sympy.physics.mechanics as me


O = Point('O')
P1 = Point('P1')
P2 = Point('P2')
P3 = Point('P3')
Xc = Point('Xc')
Xd = Point('Xd')

q1, q2, q3, q4, q5, q6 = dynamicsymbols('q1, q2, q3, q4, q5, q6', real=True, positive=True)
u1, u2, u3, u4, u5, u6 = dynamicsymbols('u1, u2, u3, u4, u5, u6')
alpha_z1, beta_z1 = dynamicsymbols('alpha_z1 beta_z1')
alpha_x1, beta_x1 = dynamicsymbols('alpha_x1 beta_x1')
alpha_y1, beta_y1 = dynamicsymbols('alpha_y1 beta_y1')
x_0 = symbols('x_0', real=True, positive=True)
ixx1, iyy1, izz1 = symbols('ixx1 iyy1 izz1')
M,g,t = symbols('M g t')
omega_z1 = dynamicsymbols('omega_z1')
omega_x1 = dynamicsymbols('omega_x1')
omega_y1 = dynamicsymbols('omega_y1')
L = symbols('L', real = True, positive=True)
T1 = symbols('T1')
k1 = symbols('k1')
delta_l1 = symbols('delta_l1')

N = ReferenceFrame('N')
Na = ReferenceFrame('Na')

Na.orient_axis(N, 0, N.z)


P1.set_vel(frame=N, value=0)
Xc.set_pos(P1, q2*N.z - q3*N.y)
Xc.set_vel(N, u2*N.z + u3*N.y)

Xd.set_pos(Xc, -q2*N.z + q3*N.y)



ceiling = Body('c', masscenter=P1, frame=N)
p2body = Body('p2body', masscenter=P2, frame=Na)

body_inertia = inertia(Na, ixx1, 0, 0)
block = Body('block', mass=M, masscenter=Xc, frame=Na, central_inertia=body_inertia)
#block.frame.set_ang_vel(N, omega_x1*N.x)

from sympy import atan2
angle_spring = atan2(q2, q3)

rev1 = SphericalJoint(name='p1p2', parent=ceiling, child=p2body, parent_point=P1, child_point=P2, amounts=[angle_spring,0,0], coordinates=[alpha_x1,alpha_y1, alpha_z1],
                      speeds=[omega_x1, omega_y1, omega_z1],
                     rot_order='XYZ')

#Xc.pos_from(P2)

rev2 = PrismaticJoint('J2', parent=p2body, child=block, parent_point=P2, child_point=Xc, coordinates=q4, speeds=u4,
                      joint_axis=-Na.y)

Na.orient_axis(N, angle_spring, N.x)

Xc.pos_from(P2)
T1 = k1*delta_l1
T1 = T1.subs({delta_l1:  q4 - x_0}).subs({q4 : smp.sqrt(q2**2 + q3**2)})

block.frame.set_ang_vel(N, omega_x1*N.x)



block.apply_force(-M*g*N.y + T1*Na.y, point=Xc, reaction_body=p2body, reaction_point=P2)

method = JointsMethod(block, rev2)
method.form_eoms()

Matrix([[M*g/sqrt(q2(t)**2/q3(t)**2 + 1) - M*Derivative(u4(t), t) - k1*(-x_0 + sqrt(q2(t)**2 + q3(t)**2))]])

In [189]:
smp.sqrt(q2**2 + q3**2)

sqrt(q2(t)**2 + q3(t)**2)

In [190]:
rev2.kdes[0]

u4(t) - Derivative(q4(t), t)

In [206]:

constraint = Matrix([Xc.pos_from(P1).magnitude() - q4])
vel_constraint = constraint.diff(t).subs({q2.diff(t):u2, q3.diff(t):u3, q4.diff(t):u4, q2**2 + q3**2:q4**2})



kane = me.KanesMethod(N, q_ind=[ q2, q3], q_dependent=[q4], u_dependent=[u4], configuration_constraints=constraint, velocity_constraints=vel_constraint,

                      u_ind = [ u2, u3],

                      kd_eqs=[
                              q2.diff(t) - u2, q3.diff(t) + u3, rev2.kdes[0]
                              ])
kaneeq = kane.kanes_equations([block], block.loads)
#kaneeq = kane.kanes_equations(method.bodies, method.loads)

In [192]:
constraint

Matrix([[sqrt(q2(t)**2 + q3(t)**2) - q4(t)]])

In [193]:
kaneeq[0]

Matrix([
[k1*(sqrt(q2(t)**2 + q3(t)**2) - x_0(t))*q2(t)/(sqrt(q2(t)**2/q3(t)**2 + 1)*q3(t))],
[       -M*g + k1*(sqrt(q2(t)**2 + q3(t)**2) - x_0(t))/sqrt(q2(t)**2/q3(t)**2 + 1)]])

In [168]:
kaneeq[1]

Matrix([
[-M*Derivative(u2(t), t)],
[-M*Derivative(u3(t), t)]])

In [169]:
kane.q # q2 and q3 are independent

Matrix([
[q2(t)],
[q3(t)],
[q4(t)]])

In [159]:
kaneeq[0][0].subs({q4 : smp.sqrt(q2**2 + q3**2)}).diff(q2).subs({q2:0, x_0: q3 -  M*g/k1, q3:L}).subs({q3:L}) #iff(q2).subs({q2:0, L: M*g/k1 + x_0}) # this is k*delta_l*sin(theta)/(M) along the z axis, this is what we get from Lagrange Eqns

M*g/L

In [160]:
kaneeq[0][1].subs({q4 : smp.sqrt(q2**2 + q3**2)}).diff(q3).subs({q2:0, x_0: q3 -  M*g/k1, q3:L}).subs({q3:L})

k1

In [143]:
kaneeq[0][1]

-M*g + k1*(q4(t) - x_0(t))/sqrt(q2(t)**2/q3(t)**2 + 1)

In [144]:
kaneeq[0][1].diff(q3) # this is k*delta_l*cos(theta)/(M) - M*g along the y axis, this is what we get from Lagrange Eqns

k1*(q4(t) - x_0(t))*q2(t)**2/((q2(t)**2/q3(t)**2 + 1)**(3/2)*q3(t)**3)

In [207]:
kanel = kane.to_linearizer()
A, B = kanel.linearize(A_and_B=True, op_point={q2:0, u2:0, u3:0, q3:L,  x_0: L -  M*g/k1 })

In [208]:
kanel.q_i

Matrix([
[q2(t)],
[q3(t)]])

In [209]:
kanel.f_v

Matrix([[q2(t)*u2(t)/q4(t) + q3(t)*u3(t)/q4(t) - u4(t)]])

In [210]:
A.simplify()

In [211]:
A

Matrix([
[  0,    0, 1,  0],
[  0,    0, 0, -1],
[g/L,    0, 0,  0],
[  0, k1/M, 0,  0]])

In [212]:
B

Matrix(0, 0, [])

In [213]:
kanel.r # this is the input

Matrix(0, 0, [])